## Certified privacy- and unlearning-safe training on the OCT-MNIST dataset

Run over different privacy and unlearning parameters and plot the results

In [29]:
%load_ext autoreload
%autoreload 2
import os
import sys
import opacus
import copy
import logging
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import abstract_gradient_training as agt
from abstract_gradient_training import unlearning
from abstract_gradient_training import AGTConfig
from abstract_gradient_training import certified_training_utils as ct_utils
from abstract_gradient_training import test_metrics

sys.path.append('..')
from models.deepmind import DeepMindSmall 
from datasets import oct_mnist

# opacus doesn't respect my logging handler :(
logger = logging.getLogger("abstract_gradient_training")
logger.handlers.clear()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def get_epsilon_delta(
    config: agt.AGTConfig,
    dl_train: torch.utils.data.DataLoader,
    delta: float = 10e-5,
) -> float:
    """
    For the given model and abstract gradient training config, compute the equivalent epsilon and delta values using
    opacus.

    Args:
        model (torch.nn.Sequential): Neural network model. Must be a torch.nn.Sequential object with dense layers and
            ReLU activations only. The model may have other layers (e.g. convolutional layers) before the dense section,
            but these must be fixed and are not trained. If fixed non-dense layers are provided, then the transform
            function must be set to propagate bounds through these layers.
        config (AGTConfig): Configuration object for the abstract gradient training module. See the configuration module
            for more details.
        dl_train (DataLoader): Training data loader.
        delta (float, optional): Desired delta value for the privacy calculation. Defaults to 10e-5.

    Returns:
        float: The epsilon value for the given model and config.
    """

    # get the config variables
    device = config.device
    lr_decay = config.lr_decay
    lr_min = config.lr_min
    dp_sgd_sigma = config.dp_sgd_sigma
    clipping = config.clip_gamma
    learning_rate = config.learning_rate
    n_epochs = config.n_epochs
    batchsize = next(iter(dl_train))[0].size(0)
    model = DeepMindSmall(1, 1).to(device)

    if config.loss == "binary_cross_entropy":
        criterion = torch.nn.BCELoss()
    else:
        raise NotImplementedError(f"Loss function {config.loss} not implemented for eps-delta calculation.")

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    privacy_engine = opacus.PrivacyEngine(accountant="rdp")
    model_private, optimizer_private, data_loader_private = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=dl_train,
        noise_multiplier=dp_sgd_sigma,
        max_grad_norm=clipping,
        poisson_sampling=False,
    )

    def get_lr(epoch):
        lr = max(1 / (1 + lr_decay * epoch), lr_min / learning_rate)
        return lr

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer_private, get_lr)

    for _ in range(n_epochs):
        for i, (x, u) in enumerate(dl_train):
            # AGT only takes full batches
            if u.size(0) < batchsize:
                break
            u, x = u.to(device), x.to(device)
            output = model_private(x)
            loss = criterion(output.squeeze().float(), u.squeeze().float())
            # Backward and optimize
            optimizer_private.zero_grad()
            loss.backward()
            optimizer_private.step()
            scheduler.step()

    # compute privacy guarantees
    epsilon = privacy_engine.accountant.get_epsilon(delta=delta)
    return epsilon


In [3]:
# set plotting options
sns.set_theme(context="poster", style="whitegrid", font_scale=1.7)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
palette = ["#DDC4DD", "#DCCFEC", "#A997DF", "#4F517D", "#1A3A3A"][::-1]
fontsize = "large"
seed = 2
results_dir = ".results/"
notebook_id = f"oct_sweep_v4_{seed}"
model_path = ".models/medmnist.ckpt"  # pretrained model path
draft = True  # whether to compute the full suite of results or a quicker reduced version
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

### Define the nominal config, model and dataloaders

In [7]:
batchsize = 5000
nominal_config = AGTConfig(
    fragsize=2000,
    learning_rate=0.2,
    n_epochs=2,
    forward_bound="interval",
    device="cuda:1",
    backward_bound="interval",
    loss="binary_cross_entropy",
    log_level="DEBUG",
    lr_decay=2.0,
    # dp_sgd_sigma=1.0,
    lr_min=0.001,
    early_stopping=False,
    metadata=f"model={model_path}"
)


# get the "DeepMindSmall" model, pretrained on the MedMNIST dataset (without class 2, Drusen)
model = DeepMindSmall(1, 1)
model.load_state_dict(torch.load(model_path))
model = model.to(nominal_config.device)

# get dataloaders, train dataloader is a mix of drusen and the "healthy" class
dl_train, _ = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1], balanced=True)
_, dl_test_drusen = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[0, 1, 3])
_, dl_test_other = oct_mnist.get_dataloaders(batchsize, 1000, exclude_classes=[2])
_, dl_test_all = oct_mnist.get_dataloaders(batchsize, 1000)

In [43]:
# compute pre-trained model accuracy
pretrain_acc = test_metrics.test_accuracy(
    *ct_utils.get_parameters(model), *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers
)[1]
pretrain_acc_all = test_metrics.test_accuracy(
    *ct_utils.get_parameters(model), *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
)[1]
pretrain_acc_other = test_metrics.test_accuracy(
    *ct_utils.get_parameters(model), *next(iter(dl_test_other)), model, ct_utils.propagate_conv_layers
)[1]
print(f"Pre-trained accuracy on Drusen: {pretrain_acc:.3g}", file=sys.stderr)
print(f"Pre-trained accuracy on all classes:  {pretrain_acc_all:.3g}", file=sys.stderr)
print(f"Pre-trained accuracy on classes 0, 1, 3:  {pretrain_acc_other:.3g}", file=sys.stderr)

# perform one certified training run with to check the nominal accuracy we get
config = copy.deepcopy(nominal_config)
config.k_private = 1
config.clip_gamma = 0.5
config.dp_sgd_sigma = 0.1
torch.manual_seed(seed)
param_l, param_n, param_u = agt.privacy_certified_training(
    model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
)

# compute the accuracies of the fine-tuned model
finetune_acc = test_metrics.test_accuracy(
    param_l, param_n, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers
)[1]
finetune_acc_all = test_metrics.test_accuracy(
    param_l, param_n, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
)[1]
finetune_acc_other = test_metrics.test_accuracy(
    param_l, param_n, param_u, *next(iter(dl_test_other)), model, ct_utils.propagate_conv_layers
)[1]
print(f"Fine-tuned accuracy on Drusen: {finetune_acc:.3g}",file=sys.stderr)
print(f"Fine-tuned accuracy on all classes: {finetune_acc_all:.3g}",file=sys.stderr)
print(f"Fine-tuned accuracy on classes 0, 1, 3: {finetune_acc_other:.3g}", file=sys.stderr)

percent_certified = test_metrics.proportion_certified(
    param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
)
print(f"Fine-tuned percent certified on all classes: {percent_certified:.3g}", file=sys.stderr)

del param_l, param_n, param_u
torch.cuda.empty_cache()

print(get_epsilon_delta(config, dl_train, delta=10e-5))

Pre-trained accuracy on Drusen: 0.456
Pre-trained accuracy on all classes:  0.835
Pre-trained accuracy on classes 0, 1, 3:  0.961
09/28/2024 03:51:19:INFO:=================== Starting Privacy Certified Training ===================
09/28/2024 03:51:19:DEBUG:	Optimizer params: n_epochs=2, learning_rate=0.2, l1_reg=0.0, l2_reg=0.0
09/28/2024 03:51:19:DEBUG:	Learning rate schedule: lr_decay=2.0, lr_min=0.001, early_stopping=False
09/28/2024 03:51:19:DEBUG:	Privacy parameter: k_private=1
09/28/2024 03:51:19:DEBUG:	Clipping: gamma=0.5, method=clamp
09/28/2024 03:51:19:DEBUG:	Noise: type=gaussian, sigma=0.1
09/28/2024 03:51:19:DEBUG:	Bounding methods: forward=interval, loss=binary_cross_entropy, backward=interval
09/28/2024 03:51:19:DEBUG:	Using Gaussian privacy-preserving noise (std 0.05)
09/28/2024 03:51:19:INFO:Starting epoch 1
09/28/2024 03:51:19:DEBUG:Initialising dataloader batchsize to 5000
09/28/2024 03:51:19:INFO:Training batch 1: Network eval bounds=(0.46, 0.46, 0.46), W0 Bound=0.0 

328.0724533098246


/vol/bitbucket/ps1623/venv/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the smallest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


In [10]:
def run_with_config(config):
    """If results for this configuration are already computed, load them from disk. Otherwise, run the certified
    training using AGT, then save and return the results."""
    fname = f"{results_dir}/{notebook_id}_{config.hash()}"
    if os.path.isfile(fname):  # run exists, so return the previous results
        param_l, param_n, param_u = torch.load(fname)
    else:
        # check whether the given config should be either unlearning or privacy training
        assert not (config.k_unlearn and config.k_private)
        torch.manual_seed(seed)
        if config.k_private:
            param_l, param_n, param_u = agt.privacy_certified_training(
                model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
            )
        else:
            param_l, param_n, param_u = agt.unlearning_certified_training(
                model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
            )
        torch.save((param_l, param_n, param_u), fname)
    # get nominal accuracy (on the Drusen class) and percent certified (on the entire test set)
    param_l = [p.to(nominal_config.device) for p in param_l]
    param_n = [p.to(nominal_config.device) for p in param_n]
    param_u = [p.to(nominal_config.device) for p in param_u]
    return param_l, param_n, param_u

In [20]:
clip_gammas = [0.5, 1.0, 2.0]

privacy_runs = {}

for gamma in clip_gammas:
    config = copy.deepcopy(nominal_config)
    config.k_private = 1
    config.clip_gamma = gamma
    config.dp_sgd_sigma = 0.0
    privacy_runs[gamma] = run_with_config(config)

    # epsilons[sigma] = get_epsilon_delta(config, dl_train, delta=10e-5)

In [42]:
epsilons = [10.0, 1.0, 0.5]
n_runs = 100


for epsilon in epsilons:
    # print(f"============= {epsilon=} ===============")

    for gamma, (param_l, param_n, param_u) in privacy_runs.items():
        # print accuracy
        percent_certified = test_metrics.proportion_certified(
            param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
        )
        private_accs = []
        # avg over 100 runs
        for _ in range(100):
            private_accs.append(
                test_metrics.test_accuracy(
                    param_l, param_n, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers, noise_level=1 / epsilon
                )[1]
            )
        mean_private_acc = np.mean(private_accs)
        std_private_acc = np.std(private_accs)
        
        print(f"\t {epsilon} &  {gamma} & {100 * mean_private_acc:.1f} $\pm$ {100 * std_private_acc:.2g} & {100*percent_certified:.1f} & {epsilon * (1 - percent_certified):.3g}\\\\")

	 10.0 &  0.5 & 88.1 $\pm$ 0.18 & 99.7 & 0.03\\
	 10.0 &  1.0 & 87.3 $\pm$ 0.19 & 98.1 & 0.19\\
	 10.0 &  2.0 & 87.2 $\pm$ 0.16 & 97.0 & 0.3\\
	 1.0 &  0.5 & 65.1 $\pm$ 1.3 & 99.7 & 0.003\\
	 1.0 &  1.0 & 64.6 $\pm$ 1.5 & 98.1 & 0.019\\
	 1.0 &  2.0 & 64.5 $\pm$ 1.4 & 97.0 & 0.03\\
	 0.5 &  0.5 & 58.6 $\pm$ 1.6 & 99.7 & 0.0015\\
	 0.5 &  1.0 & 58.4 $\pm$ 1.6 & 98.1 & 0.0095\\
	 0.5 &  2.0 & 58.2 $\pm$ 1.6 & 97.0 & 0.015\\


## DP-SGD

In [52]:
from tqdm import trange

# perform one certified training run with to check the nominal accuracy we get
config = copy.deepcopy(nominal_config)
config.k_private = 1
config.clip_gamma = 1.0
config.dp_sgd_sigma = 1.0
config.log_level = "WARNING"
print("epsilon=", get_epsilon_delta(config, dl_train, delta=10e-5), "clip_gamma", config.clip_gamma)

private_accs = []
n_runs = 50

for _ in trange(n_runs):
    param_l, param_n, param_u = agt.privacy_certified_training(
        model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
    )
    private_accs.append(
        test_metrics.test_accuracy(
            param_l, param_n, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
        )[1]
    )

mean_private_acc = np.mean(private_accs)
std_private_acc = np.std(private_accs)

print(100 * mean_private_acc, 100 * std_private_acc)

epsilon= 4.7142437673284485 clip_gamma 1.0


100%|██████████| 50/50 [04:12<00:00,  5.06s/it]

85.97600424289703 1.280400118718563


In [53]:
from tqdm import trange

# perform one certified training run with to check the nominal accuracy we get
config = copy.deepcopy(nominal_config)
config.k_private = 1
config.clip_gamma = 2.0
config.dp_sgd_sigma = 1.0
config.log_level = "WARNING"
print("epsilon=", get_epsilon_delta(config, dl_train, delta=10e-5), "clip_gamma", config.clip_gamma)

private_accs = []
n_runs = 50

for _ in trange(n_runs):
    param_l, param_n, param_u = agt.privacy_certified_training(
        model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
    )
    private_accs.append(
        test_metrics.test_accuracy(
            param_l, param_n, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
        )[1]
    )

mean_private_acc = np.mean(private_accs)
std_private_acc = np.std(private_accs)

print(100 * mean_private_acc, 100 * std_private_acc)

/vol/bitbucket/ps1623/venv/lib/python3.10/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/vol/bitbucket/ps1623/venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


epsilon= 4.7142437673284485 clip_gamma 2.0


100%|██████████| 50/50 [04:12<00:00,  5.06s/it]

81.89000368118286 1.6191668870743088


In [54]:
from tqdm import trange

# perform one certified training run with to check the nominal accuracy we get
config = copy.deepcopy(nominal_config)
config.k_private = 1
config.clip_gamma = 1.0
config.dp_sgd_sigma = 0.1
config.log_level = "WARNING"
print("epsilon=", get_epsilon_delta(config, dl_train, delta=10e-5), "clip_gamma", config.clip_gamma)

private_accs = []
n_runs = 50

for _ in trange(n_runs):
    param_l, param_n, param_u = agt.privacy_certified_training(
        model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
    )
    private_accs.append(
        test_metrics.test_accuracy(
            param_l, param_n, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
        )[1]
    )

mean_private_acc = np.mean(private_accs)
std_private_acc = np.std(private_accs)

print(100 * mean_private_acc, 100 * std_private_acc)

/vol/bitbucket/ps1623/venv/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the smallest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


epsilon= 328.0724533098246 clip_gamma 1.0


100%|██████████| 50/50 [04:12<00:00,  5.05s/it]

87.5560040473938 0.303420978474976


In [55]:
from tqdm import trange

# perform one certified training run with to check the nominal accuracy we get
config = copy.deepcopy(nominal_config)
config.k_private = 1
config.clip_gamma = 2.0
config.dp_sgd_sigma = 0.1
config.log_level = "WARNING"
print("epsilon=", get_epsilon_delta(config, dl_train, delta=10e-5), "clip_gamma", config.clip_gamma)

private_accs = []
n_runs = 50

for _ in trange(n_runs):
    param_l, param_n, param_u = agt.privacy_certified_training(
        model, config, dl_train, dl_test_drusen, transform=ct_utils.propagate_conv_layers
    )
    private_accs.append(
        test_metrics.test_accuracy(
            param_l, param_n, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers
        )[1]
    )

mean_private_acc = np.mean(private_accs)
std_private_acc = np.std(private_accs)

print(100 * mean_private_acc, 100 * std_private_acc)

epsilon= 328.0724533098246 clip_gamma 2.0


100%|██████████| 50/50 [04:12<00:00,  5.06s/it]

87.3320038318634 0.4105803520599861


epsilon = 328.0, clip = 0.5, acc = 88.1% +- 0.23%
epsilon = 328.0, clip = 1.0, acc = 87.6% +- 0.30%
epsilon = 328.0, clip = 2.0, acc = 87.3 +- 0.41

epsilon = 4.71, clip = 0.5, acc = 87.5 +- 0.66
epsilon = 4.71, clip = 1.0, acc = 85.98 +- 1.28
epsilon = 4.71, clip = 2.0, acc = 81.89 +- 1.62

epsilon = 1.53, clip = 0.5, acc = 85.72 +- 1.51
epsilon = 1.53, clip = 1.0, acc = 81.5 +- 2.04
epsilon = 1.53, clip = 2.0, acc = 74.1, +- 3.60